In [2]:
import json
import nltk
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
from gensim.models import KeyedVectors
import os
from nltk.tokenize import word_tokenize

from src.core import file_manager as fm

2022-06-23 15:49:01.719528: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-23 15:49:01.719544: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Sentences

In [3]:
df = fm.read_annotated_df_with_embeddings('bert_pt')

count_before = df.txt.count()

In [4]:
df = df.loc[df.apply(lambda x: (len(x['tokens']) == len(x['word_embeddings'])) , axis=1)]

rows_delted = count_before - df.txt.count()

print(f'There was deleted {rows_delted} rows with bad representation')

There was deleted 21 rows with bad representation


In [7]:
vocab = {}

for index, row in df.iterrows():
  for index, token in enumerate(row['tokens']):
    if token not in vocab:
      vocab[token] = np.array(row['word_embeddings'][index])

n_source_words = len(vocab)

print(n_source_words)

6231


In [14]:
keys_lower = {}

for key in vocab.keys():
  if not key.lower() in keys_lower:
    keys_lower[key.lower()] = []

  keys_lower[key.lower()].append(key)


keys_lower

{'tô': ['Tô', 'tô'],
 'muito': ['muito', 'Muito', 'MUITO'],
 'nervosa': ['nervosa'],
 'marcela': ['Marcela'],
 'com': ['com', 'Com', 'COM'],
 'tudo': ['tudo', 'Tudo'],
 'isso': ['isso', 'Isso', 'ISSO'],
 'medo': ['medo', 'Medo', 'MEDO'],
 'não': ['Não', 'não', 'NÃO'],
 'tem': ['Tem', 'tem', 'TEM'],
 'algum': ['algum', 'Algum'],
 'remédio': ['remédio', 'Remédio'],
 'que': ['que', 'Que'],
 'a': ['a', 'A'],
 'pessoa': ['pessoa'],
 'tome': ['tome'],
 'para': ['para', 'Para'],
 'se': ['se', 'Se', 'SE'],
 'acalmar': ['acalmar'],
 'tá': ['Tá', 'tá'],
 'certo': ['certo', 'Certo'],
 'bom': ['bom', 'Bom'],
 'obrigada': ['Obrigada', 'obrigada', 'OBRIGADA'],
 ',': [','],
 'deus': ['Deus', 'deus', 'DEUS'],
 'te': ['te', 'Te'],
 'abençoe': ['abençoe'],
 'eu': ['Eu', 'eu', 'EU'],
 'preciso': ['preciso', 'Preciso'],
 'de': ['de', 'De', 'DE'],
 'um': ['um', 'Um', 'UM'],
 'psicólogo': ['psicólogo'],
 'pra': ['pra', 'Pra'],
 'eles': ['eles', 'Eles'],
 'o': ['O', 'o'],
 'luto': ['luto'],
 'é': ['é', 'É'],

In [10]:
vocab['bom'][:10]

array([-0.15446484, -0.59753007, -0.12694153,  0.31093362,  0.44728976,
        0.1666815 , -0.05694415, -0.01199269,  0.24765942, -0.6515901 ])

In [15]:
keys_with_variations ={key: value for key, value in keys_lower.items() if len(value) > 1}

for key in keys_with_variations.keys():
  first_key = keys_with_variations[key][0]
  
  for index in range(1, len(keys_with_variations[key])):
    key_to_compare = keys_with_variations[key][index]
    
    if not (vocab[first_key] == vocab[key_to_compare]).all():
      print(f'There is a diference between: {first_key} and {key_to_compare} ')

There is a diference between: Tô and tô 
There is a diference between: muito and Muito 
There is a diference between: muito and MUITO 
There is a diference between: com and Com 
There is a diference between: com and COM 
There is a diference between: tudo and Tudo 
There is a diference between: isso and Isso 
There is a diference between: isso and ISSO 
There is a diference between: medo and Medo 
There is a diference between: medo and MEDO 
There is a diference between: Não and não 
There is a diference between: Não and NÃO 
There is a diference between: Tem and tem 
There is a diference between: Tem and TEM 
There is a diference between: algum and Algum 
There is a diference between: remédio and Remédio 
There is a diference between: que and Que 
There is a diference between: a and A 
There is a diference between: para and Para 
There is a diference between: se and Se 
There is a diference between: se and SE 
There is a diference between: Tá and tá 
There is a diference between: cert

In [18]:
vocab = {}

vocab['unknown'] = np.zeros(len(df.loc[0].word_embeddings[0]))

for index, row in df.iterrows():
  for index, token in enumerate(row['tokens']):
    token_lower = token.lower()
    word_embedding = np.array(row['word_embeddings'][index])
    
    if token_lower not in vocab and np.any(word_embedding):
      vocab[token_lower] = word_embedding 


n_source_words = len(vocab)

print(n_source_words)

5362


In [19]:
df['intent'].value_counts()

inform_symptoms    5474
request_inform     1278
greeting           1175
inform_medicine     889
Name: intent, dtype: int64

In [20]:
max_len = df['tokens'].apply(lambda x : len(x)).max()
print(max_len)
# max_len = 700 #75% do dataset pelo menos tem no maximo esse valor de tamanho

103


In [21]:
def create_embedding_matrix(vocab, n_words, emb_size):
    
    pret_embedding = {}
    embedding_matrix = np.zeros((n_words, emb_size))
    count = 0

    # Dicionário com todos os ids e palavras do embedding pré-treinado
    # for index, word in enumerate(vocab.index_to_key):
    for index, word in enumerate(vocab.keys()):
      pret_embedding[word] = index

    # Construindo a embedding_matrix do embedding pré-treinado
    for item in pret_embedding.items():
      if item[1] < n_words:
        count += 1
        embedding_vector = vocab[item[0]]
        if embedding_vector is not None:
          embedding_matrix[item[1]] = embedding_vector
      
    return embedding_matrix

## Save the vectors  in a new Matrix

In [22]:
# embedding_size = 300
embedding_size = len(df.loc[0].word_embeddings[0])
embedding_matrix = create_embedding_matrix(vocab, n_source_words, embedding_size)
embedding_matrix.shape

(5362, 768)

In [23]:
vocab_key_2_index =  {key: index for index, key in enumerate(vocab.keys())}

## Padding

In [24]:
# Função que cria o X
def create_x(tokens):
    list_x = []
    for token in tokens:
      if(token in vocab_key_2_index):
        list_x.append(vocab_key_2_index[token])
      
    return list_x


In [25]:
df['token_indexes'] = df['tokens'].apply(lambda tokens : create_x(tokens))
df.head(5)

,txt,annotated_txt,intent,embeddings,tokens,word_embeddings,token_indexes
0,Tô muito nervosa Marcela com tudo isso,Tô muito nervosa Marcela com tudo isso,inform_symptoms,"[[-0.17518604000000002, -0.38116214, 0.1568575...","[Tô, muito, nervosa, Marcela, com, tudo, isso]","[[0.10563439000000001, -0.6496529600000001, 0....","[2, 3, 5, 6, 7]"
1,Com muito medo,Com muito medo,inform_symptoms,"[[0.40117407, -0.22014676, 0.1467775, 0.185312...","[Com, muito, medo]","[[0.5744509, -0.58911455, 0.11015732600000001,...","[2, 8]"
2,Não,Não,greeting,"[[0.0857483, -0.41115219999999997, 0.28060192,...",[Não],"[[0.0857483, -0.41115219999999997, 0.28060192,...",[]
3,Tem algum remédio que a pessoa tome para se a...,Tem algum remédio que a pessoa tome para se a...,request_inform,"[[-0.018409189, -0.17020282, 0.273364780000000...","[Tem, algum, remédio, que, a, pessoa, tome, pa...","[[-0.026540479000000002, -0.44967073, 0.199957...","[11, 12, 13, 14, 15, 16, 17, 18, 19]"
4,Tá certo,Tá certo,greeting,"[[0.2476745, -0.31360537, 0.040109605, 0.37392...","[Tá, certo]","[[0.29746079999999997, -0.3857502, 0.25457925,...",[21]


In [26]:
X = pad_sequences(maxlen=max_len, sequences=df['token_indexes'], value=0, padding='post', truncating='post')
Y = pd.get_dummies(df['intent']).values
(X.shape, Y.shape)

((8816, 103), (8816, 4))

## Create train and test

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42,stratify=Y)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7052, 103) (7052, 4)
(1764, 103) (1764, 4)


## LSTM Model

In [28]:
num_labels = Y.shape[1]
MAX_NB_WORDS = n_source_words
# EMBEDDING_DIM = 50
EMBEDDING_DIM = embedding_size

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1], weights=[embedding_matrix]))
# model.add(Embedding(X.shape[0], X.shape[1], input_length=X.shape[1], weights=[X]))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.1))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#Optimisation functions usually calculate the gradient i.e. the partial derivative of loss function with respect to weights, 
#and the weights are modified in the opposite direction of the calculated gradient.

2022-06-23 16:39:22.860433: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 16:39:22.861824: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-23 16:39:22.862005: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-23 16:39:22.862121: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-23 16:39:22.862327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 103, 768)          4118016   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              426496    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 4)                 516       
                                                                 
Total params: 4,545,028
Trainable params: 4,545,028
Non-trainable params: 0
_________________________________________________________________


In [30]:
path_model = fm.filename_from_data_dir(f'output/patient/lstm_models/bert_pt.h5')
#model.load_weights(path_model)

In [31]:
epochs = 20
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/20
100/100 [==============================] - 16s 146ms/step - loss: 0.5078 - accuracy: 0.8046 - val_loss: 0.3136 - val_accuracy: 0.8711
Epoch 2/20
100/100 [==============================] - 14s 138ms/step - loss: 0.2479 - accuracy: 0.8993 - val_loss: 0.2640 - val_accuracy: 0.8952
Epoch 3/20
100/100 [==============================] - 14s 140ms/step - loss: 0.1800 - accuracy: 0.9335 - val_loss: 0.2813 - val_accuracy: 0.8824
Epoch 4/20
100/100 [==============================] - 14s 142ms/step - loss: 0.1387 - accuracy: 0.9488 - val_loss: 0.2975 - val_accuracy: 0.9051
Epoch 5/20
100/100 [==============================] - 14s 142ms/step - loss: 0.1158 - accuracy: 0.9575 - val_loss: 0.3002 - val_accuracy: 0.8952


In [32]:
working_dir = fm.filename_from_data_dir(f'output/lstm_models/patient/bert_pt')

In [33]:
print('saving the model')
model.save(f'{working_dir}/model.h5')

saving the model


In [34]:
print('saving the vocabullary')

file = open(f'{working_dir}/vocabullary.json',"w")

file.write(json.dumps(vocab_key_2_index))

file.close()

saving the vocabullary


In [35]:
print('saving the metadata')

file = open(f'{working_dir}/metadata.json',"w")

intents = pd.get_dummies(df['intent']).columns.tolist()

metadata = {
  'intents': intents,
  'vector_length': str(max_len)
}

file.write(json.dumps(metadata))

file.close()

saving the metadata


## Evaluate the model

In [36]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

56/56 [==============================] - 1s 19ms/step - loss: 0.2654 - accuracy: 0.8997
Test set
  Loss: 0.265
  Accuracy: 0.900
